In [2]:
#Import necessary packages
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder,DeployTransactionBuilder,MessageTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
import requests
import json
import pickle
import csv
import os
from repeater import retry
from iconsdk.libs.in_memory_zip import gen_deploy_data_content

In [3]:
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
SCORE_ADDRESS = "cxf145f42657dbf2f98f02427b55e27c67d2951220"
NEWTON_ADDRESS = "hx9393939393939393939393939393939393939399"
DEPLOY_PARAMS = {}

In [4]:
#Start ICON service and wallet
icon_service = IconService(HTTPProvider("https://bicon.net.solidwallet.io", 3))
#icon_service = IconService(HTTPProvider("http://localhost:9000", 3))
operator = KeyWallet.load('config/yeouido/keystores/operator', 'bgsa199@#')
prajwol = KeyWallet.load('config/yeouido/keystores/testPrajwol', 'bgsa199@#')



In [62]:
def test_method(method_name:str, params:dict = {}):
    call_transaction = CallTransactionBuilder()\
    .from_(operator.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .method(method_name)\
    .params(params)\
    .build()

    estimate_step = icon_service.estimate_step(call_transaction)
    step_limit = estimate_step + 100000
    signed_transaction = SignedTransaction(call_transaction, operator, step_limit)

    tx_hash = icon_service.send_transaction(signed_transaction)
    return tx_hash

def update(name, symbol, supply, decimals=18):
    UPDATE_PARAMS =  {
            "_tokenName": name,
            "_symbolName": symbol,
			"_initialSupply": supply,
            "_decimals": decimals,
            "_cap":1000
		}
        
    deploy_transaction = DeployTransactionBuilder()\
    .from_(operator.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('ODIContracts'))\
    .params(UPDATE_PARAMS)\
    .build()

    estimate_step = icon_service.estimate_step(deploy_transaction)
    step_limit = estimate_step + 100000
    signed_transaction = SignedTransaction(deploy_transaction, operator, step_limit)    
    tx_hash = icon_service.send_transaction(signed_transaction)
    return tx_hash

def call_method(method_name, params={}):
    call = CallBuilder().from_(operator.get_address())\
                    .to(SCORE_ADDRESS)\
                    .method(method_name)\
                    .params(params)\
                    .build()
    return icon_service.call(call)

def name_and_symbol():
    external_methods = ["name", "symbol"]
    for method in external_methods:
        print(call_method(method))

def total_supply():
    total_supply_in_hex = call_method('totalSupply')
    return(int(total_supply_in_hex, 16))

def balance_of(address):
    hex = call_method('balanceOf', {'account':address})
    return int(hex, 16)
    
def mint(amount):
    test_method('mint', {'_amount':amount})

def burn(amount):
    test_method('burn', {'_amount':amount})

def mintTo(account, amount):
    test_method('mintTo', {'_account':account, '_amount':amount})

def transfer(to, amount):
    test_method('transfer', {'_to':to, '_value': amount})

def pause():
    test_method('pause',{})

def unpause():
    test_method('unpause',{})

def paused():
    print(call_method('paused'))

def cap():
    hex = call_method('cap', {})
    return int(hex, 16)


In [71]:
txhash = update('3estToken', '3TK', 1100)
txhash

'0x56378b6f34e9333471a9149e4bcec43e0c66309707da5523e44469f6a5b4d0a2'

In [74]:
print(icon_service.get_transaction_result(txhash))

{'txHash': '0x56378b6f34e9333471a9149e4bcec43e0c66309707da5523e44469f6a5b4d0a2', 'blockHeight': 3768543, 'blockHash': '0x53b3ed1ca558e2fd1333e73fc1601dc95e4cb380d919d11bd85e5e06f8a4ee12', 'txIndex': 2, 'to': 'cxf145f42657dbf2f98f02427b55e27c67d2951220', 'stepUsed': 1838134200, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1838292260, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [75]:
name_and_symbol()

2estToken
2TK


In [56]:
total_supply()

1100000000000000000000

In [42]:
mint(500000000000000000000)

In [35]:
paused()

0x0


In [40]:
unpause()

In [53]:
cap()

Raised JSONRPCException while returning the custom response. Error message: {'code': -32003, 'message': 'Method not found: SampleToken.cap'}


JSONRPCException: {'code': -32003, 'message': 'Method not found: SampleToken.cap'} (Json rpc error)